In [1]:
from typing import *
import wafel
import ctypes as C
import struct
from itertools import chain, combinations, product
from random import *
import numpy as np
import copy

In [2]:
# ---general config---

# path to unlocked dll (unlock through wafel)
#game = wafel.Game('../wafel_0_8_1_win_x64/libsm64/sm64_jp.dll')
game = wafel.Game('../wafel_0_8_1_win_x64/libsm64/sm64_us.dll')
# path to initial m64
filename = 'best_found.m64'
# Output filename if an improvement is found
# Can set it to overwrite the old file if you don't mind it being overwritten
filename_new = 'best_found.m64'
# The m64 needs to be from start, but only want to brute force the joystick
# in a certain slice of time
# Note: This is 1 less than the frame number displayed in Mupen
# first affected frame is start_frame+1 (like after loading a st in mupen, you need to advance first)

#start_frame = 71901
#end_frame = 71991
start_frame = 2360
end_frame = 2388#9
#73494, 501, 508, 515, 522, 529, 536
# ---config end---

In [3]:
# Create a savestate slot
power_on = game.save_state()
m64 = wafel.load_m64(filename)


# apply current m64 data, it doesn't happen itself
def set_inputs(g, inputs):
    g.write('gControllerPads[0].button', inputs.buttons)
    g.write('gControllerPads[0].stick_x', inputs.stick_x)
    g.write('gControllerPads[0].stick_y', inputs.stick_y)

In [4]:
# Run through the m64 and make a savestate at the beginning of the bruteforce window.
# Also prints out number of stars just to make sure things are
# working/no desyncs
print(f'Fast forwarding to frame {start_frame}...')
for frame in range(len(m64[1])):
    set_inputs(game, m64[1][frame])
    game.advance()

    # print periodically
    if frame % 10000 == 0:
        print("Frame %05d stars %02d" % (frame, game.read('gMarioState.numStars')))
    if frame == start_frame:
        print("---bruteforce start---")
        game.write('gMarioState.pos[0]', 2387.1)
        game.write('gMarioState.pos[1]', 1324)
        game.write('gMarioState.pos[2]', 1757.59)
        game.write('gMarioState.faceAngle[1]', 24550)
        game.write('gMarioState.forwardVel', 48.56554)
        startst = game.save_state()
        #break
        
    # Can disable this to examine actions if you want
    if (frame >= start_frame and frame <= end_frame + 2):
       print('{0} {1}'.format(frame, game.read('gMarioState.action')))

    # Example of making a savestate
    # game.save_state(backup)
    # backupFrame = frame + 1
    # break

Fast forwarding to frame 2360...
Frame 00000 stars 00
---bruteforce start---
2360 50333830
2361 50333830
2362 50333830
2363 50333830
2364 50333830
2365 50333830
2366 50333830
2367 50333830
2368 50333830
2369 50333830
2370 50333830
2371 50333830
2372 50333830
2373 50333830
2374 50333830
2375 50333830
2376 50333830
2377 50333830
2378 50333830
2379 50333830
2380 50333830
2381 50333830
2382 50333830
2383 50333830
2384 50333830
2385 50333830
2386 50333830
2387 50333830
2388 50333830
2389 50333830
2390 50333830


In [5]:
# All of the symbols have to be found through decomp
# Example accessing objects:
# game.write('gObjectPool[3]->oForwardVel', 100) #object field names can be found in include/object_fields.h
#                         ^-- in stroop this is slot from memory order -1
# example reading some values

# marioAction = game.read('gMarioState.action')

# Make a backup of the original inputs
m64_orig = copy.copy(m64)

In [6]:
#Convert from [0, 255] to [-128, 127]
def to_actual_coord(wafel_coord):
    return wafel_coord - 256*(wafel_coord > 127)

In [7]:
# Helper function to ignore deadzones while
# incrementing a coordinate
# to decrement, can say x = -increment_coord(-x)
# !!!changed for wafel api because its unsigned here!!! seems to be a bit broken
def increment_coord(coord):
    #coord = coord - 128
    coord = to_actual_coord(coord)
    if coord == -8:
        coord = 0
    elif coord == 0:
        coord = 8
    elif coord >= 127:
        coord = 127
    else:
        coord = coord + 1
    #return coord + 128
    return coord % 256

In [8]:
# Can include this for regularization if you want. Will penalize difference
# between joystick numbers on consecutive frames
def l1ofdiff(m64):
    tot = 0
    for i in range(start_frame, end_frame + 1):
        tot += abs(to_actual_coord(m64[1][i].stick_x) -
                   to_actual_coord(m64[1][i + 1].stick_x))
        tot += abs(to_actual_coord(m64[1][i].stick_y) -
                   to_actual_coord(m64[1][i + 1].stick_y))
    return tot

In [9]:
#bomb = [0, 1329, -3583]
#bomb_x = 0
#bomb_y = 1329
#bomb_z = -3583
#misa = [-4095.99, -2047, 3072.99]
#star = [-2900, -2700, 3650]
#gp_up_point = [1586, -1717, 2356]
land_point = [3596, -3797, 3092]
kick_point = [4150, -3174, -768]
weight = [.01, .01, 1]

def opt_func():
    mario_pos = game.read('gMarioState.pos')
    hspd = game.read('gMarioState.forwardVel')
    slideyaw = (game.read('gMarioState.slideYaw')) % 65536
    slidevel_x = (game.read('gMarioState.slideVelX'))
    slidevel_z = (game.read('gMarioState.slideVelZ'))
    yspd = game.read('gMarioState.vel')[1]
    mario_yaw = game.read('gMarioState.faceAngle')[1]
    mario_action = game.read('gMarioState.action')
    
    return mario_pos[2]
    #return -mario_pos[1] - hspd + (yspd < 0)*1000 + (mario_action != 67110001)*1000
    #return (sum(((kick_point[i] - mario_pos[i])*weight[i])**2 for i in range(3)))**.5
    #return (sum((land_point[i] - mario_pos[i])**2 for i in range(3)))**.5 + abs((mario_yaw % 65536) - 59000)
    #return ((mario_pos[0]))
    #return -mario_pos[2] + (mario_pos[2] < 3845)*10
    #reach groundpound point
    #return (sum((gp_up_point[i] - mario_pos[i])**2 for i in range(3)))**.5
    #get higher on slope
    #return (-mario_pos[1])*.3 - hspd*45 + abs(mario_yaw - 24500)/30

In [ ]:
#This cell is definitely the messiest, and where you might want to do something
#different. Roughly speaking, it does annealing with random restarts.

#keep track of best m64 found and best objective value
best_ever_m64 = copy.copy(m64_orig)
best_ever_val = 9999999999

#ordered_changes = product(range(start_frame + 1, end_frame), range(0, 4), range(1, 4),
#                          range(start_frame + 1, end_frame), range(0, 4), range(1, 4))
#ordered_changes = list(ordered_changes)
#shuffle(ordered_changes)
    
for attempt in range(500000):
    #Do a random restart
    
    last_change = 0
    #Temperature for annealing
    temp = random()*.15
    #Determine the size of joystick perturbations and number of joystick perturbations
    max_changes = randint(1, 8)
    max_size = randint(1, 30)
    #With some probability, drop down to size 1 or count 1 (helps explore locally)
    big_change_num_prob = random()*.2
    big_change_size_prob = random()*.5
    m64 = (best_ever_m64[0], copy.copy(best_ever_m64[1]))
    best_val = 9999999999
    cur_val = best_val
    
    #Now try random perturbations
    for i in range(500000):
        #Break out early if these settings get stuck
        if i - last_change > 5000:
            print(temp)
            print(max_changes)
            print(max_size)
            break
        #Array to keep track of changes made to the m64.
        #Make some random changes, check fitness, revert if not good enough.
        changes = []
        if i % 1000 == 0:
            print(str(i) + ' ' + str(cur_val))
        #Lower temperature
        if i % 300 == 0:
            temp = temp*.9
        #Do a perturbation here, except for first time through
        #where we just want to get objective value
        if i > 0:
            num_changes = randint(1, max_changes)
            if random() > big_change_num_prob and i > 3000:
                    num_changes = 1
            for change in range(num_changes):
                #Randomly select change frame (with replacement in
                #the event of multiple changes)
                change_frame = randint(start_frame + 1, end_frame)
                change_dir = randint(0, 3)
                change_size = randint(1, max_size)
                if random() > big_change_size_prob and i > 3000:
                    change_size = 1
                #Make sure to save old version of frame so we can revert
                frame_old = m64[1][change_frame].copy()
                changes.append([change_frame, frame_old])
                #Modify joystick input in the appropriate size and direction
                for change in range(change_size):
                    frame_inp = m64[1][change_frame].copy()
                    # print(frame_inp.stick_y)
                    if change_dir == 0:
                        frame_inp.stick_x = increment_coord(frame_inp.stick_x)
                    elif change_dir == 1:
                        frame_inp.stick_y = increment_coord(frame_inp.stick_y)
                    elif change_dir == 2:
                        frame_inp.stick_x = 255 - increment_coord(255 - frame_inp.stick_x)
                    else:
                        frame_inp.stick_y = 255 - increment_coord(255 - frame_inp.stick_y)
                    m64[1][change_frame] = frame_inp

        #now find its fitness to minimize
        game.load_state(startst)
        offset = 0
        for frame in range(start_frame, end_frame):
            set_inputs(game, (m64[1][frame + 1]))
            game.advance()
        x, y, z = game.read('gMarioState.pos')
        hspd = game.read('gMarioState.forwardVel')
        faceyaw = game.read('gMarioState.faceAngle')[1]
        orig_fitness = opt_func()#to_hmc_bounce_before_elevator()#to_hmc_elevator()
        fitness = orig_fitness# + l1ofdiff(m64)*.002#+ l1ofdiff(m64)*5# + xnorm(m64)*7
        if fitness < cur_val:
            last_change = i
            cur_val = fitness
            if fitness < best_val:
                print('New best: {0} {1}'.format(fitness, orig_fitness))
                print('{0} {1} {2} {3} {4}'.format(x, y, z, hspd, faceyaw))
                best_val = fitness
            if fitness < best_ever_val:
                wafel.save_m64(filename_new, m64[0], m64[1])
                best_ever_val = fitness
                best_ever_m64 = (m64[0], copy.copy(m64[1]))
        #Chose the most basic function for annealing
        elif random() < np.exp(-(fitness - cur_val)/temp):
        #elif random() < np.exp(-(fitness - best_val)/temp):
            if fitness != cur_val:
                last_change = i
            cur_val = fitness
        else:
            #if we failed, revert m64
            #revert changes in reverse order of how they were made
            for change in changes[::-1]:
                m64[1][change[0]] = change[1]

0 9999999999
New best: 696.9974365234375 696.9974365234375
3267.39013671875 403.5 696.9974365234375 45.85795211791992 24550
New best: 696.9962158203125 696.9962158203125
3267.01806640625 403.5 696.9962158203125 45.8407096862793 24550
New best: 696.99267578125 696.99267578125
3266.66748046875 403.5 696.99267578125 45.8270149230957 24550
New best: 696.98681640625 696.98681640625
3266.55224609375 403.5 696.98681640625 45.81729507446289 24550
New best: 696.954833984375 696.954833984375
3266.17138671875 403.5 696.954833984375 45.72421646118164 24550
1000 697.7259521484375
2000 697.306396484375
3000 697.2215576171875
New best: 696.95166015625 696.95166015625
3257.73974609375 403.5 696.95166015625 44.85595703125 24550
New best: 696.9482421875 696.9482421875
3257.86669921875 403.5 696.9482421875 44.8603515625 24550
New best: 696.9429931640625 696.9429931640625
3257.64404296875 403.5 696.9429931640625 44.836944580078125 24550
New best: 696.9376220703125 696.9376220703125
3257.77197265625 403.5 

New best: 696.6405029296875 696.6405029296875
3257.41748046875 403.5 696.6405029296875 44.63582229614258 24550
8000 696.6756591796875
New best: 696.63720703125 696.63720703125
3258.85107421875 403.5 696.63720703125 44.75285339355469 24550
New best: 696.63525390625 696.63525390625
3258.75439453125 403.5 696.63525390625 44.73725891113281 24550
New best: 696.633056640625 696.633056640625
3258.90380859375 403.5 696.633056640625 44.741371154785156 24550
New best: 696.632568359375 696.632568359375
3258.84619140625 403.5 696.632568359375 44.75047302246094 24550
New best: 696.6317138671875 696.6317138671875
3257.74560546875 403.5 696.6317138671875 44.59786605834961 24550
New best: 696.6314697265625 696.6314697265625
3257.89404296875 403.5 696.6314697265625 44.6019172668457 24550
New best: 696.620849609375 696.620849609375
3257.03271484375 403.5 696.620849609375 44.58382034301758 24550
New best: 696.617919921875 696.617919921875
3256.91455078125 403.5 696.617919921875 44.5789909362793 24550
New

New best: 696.5657958984375 696.5657958984375
3256.93408203125 403.5 696.5657958984375 44.5928840637207 24550
17000 696.566162109375
New best: 696.5655517578125 696.5655517578125
3256.74755859375 403.5 696.5655517578125 44.58835983276367 24550
New best: 696.5650634765625 696.5650634765625
3256.59326171875 403.5 696.5650634765625 44.57938766479492 24550
New best: 696.56494140625 696.56494140625
3256.91943359375 403.5 696.56494140625 44.61624526977539 24550
18000 696.566650390625
19000 696.565185546875
New best: 696.5645751953125 696.5645751953125
3256.92333984375 403.5 696.5645751953125 44.61111831665039 24550
New best: 696.5643310546875 696.5643310546875
3256.94580078125 403.5 696.5643310546875 44.61713409423828 24550
New best: 696.5640869140625 696.5640869140625
3256.89404296875 403.5 696.5640869140625 44.60193634033203 24550
New best: 696.5638427734375 696.5638427734375
3257.00732421875 403.5 696.5638427734375 44.61255645751953 24550
20000 696.564453125
21000 696.5643310546875
22000 

17000 696.5645751953125
New best: 696.5631103515625 696.5631103515625
3256.85595703125 403.5 696.5631103515625 44.61663818359375 24550
18000 696.5633544921875
New best: 696.56298828125 696.56298828125
3256.74462890625 403.5 696.56298828125 44.603458404541016 24550
19000 696.5633544921875
20000 696.5634765625
New best: 696.5628662109375 696.5628662109375
3256.75146484375 403.5 696.5628662109375 44.60176086425781 24550
21000 696.5628662109375
New best: 696.5626220703125 696.5626220703125
3256.74658203125 403.5 696.5626220703125 44.60012435913086 24550
22000 696.5626220703125
23000 696.5626220703125
24000 696.5626220703125
25000 696.5626220703125
26000 696.5626220703125
27000 696.5626220703125
4.3310074174070966e-06
7
8
0 9999999999
New best: 696.5626220703125 696.5626220703125
3256.74658203125 403.5 696.5626220703125 44.60012435913086 24550
1000 697.4693603515625
2000 697.416748046875
3000 697.305419921875
4000 696.995361328125
5000 696.940673828125
6000 696.8095703125
7000 696.69921875


6000 696.6292724609375
7000 696.614990234375
8000 696.5875244140625
9000 696.5809326171875
10000 696.583740234375
11000 696.576416015625
12000 696.571533203125
13000 696.572265625
14000 696.567138671875
15000 696.5684814453125
16000 696.565673828125
17000 696.565185546875
18000 696.5643310546875
19000 696.5643310546875
20000 696.5643310546875
21000 696.5643310546875
22000 696.5643310546875
23000 696.5643310546875
24000 696.5643310546875
25000 696.5640869140625
26000 696.5640869140625
27000 696.5640869140625
28000 696.5640869140625
29000 696.5640869140625
1.1931263292464988e-06
2
6
0 9999999999
New best: 696.562255859375 696.562255859375
3256.66455078125 403.5 696.562255859375 44.59127426147461 24550
1000 698.1890869140625
2000 697.9166259765625
3000 697.232421875
4000 697.0498046875
5000 696.8963623046875
6000 696.7896728515625
7000 696.6678466796875
8000 696.66748046875
9000 696.6103515625
10000 696.604736328125
11000 696.603271484375
12000 696.5985107421875
13000 696.5941162109375
14

...

6000 696.6104736328125
7000 696.6461181640625
8000 696.5943603515625
9000 696.5904541015625
10000 696.57470703125
11000 696.5843505859375
12000 696.569580078125
13000 696.5709228515625
14000 696.57275390625
15000 696.5665283203125
16000 696.56591796875
17000 696.56494140625
18000 696.564697265625
19000 696.564208984375
20000 696.564208984375
21000 696.564208984375
22000 696.5634765625
23000 696.5634765625
24000 696.5634765625
25000 696.5634765625
26000 696.5634765625
3.6580931151051217e-06
4
20
0 9999999999
New best: 696.562255859375 696.562255859375
3256.66455078125 403.5 696.562255859375 44.59127426147461 24550
1000 697.0230712890625
2000 697.13427734375
3000 696.7237548828125
4000 696.7010498046875
5000 696.7220458984375
6000 696.68896484375
7000 696.6278076171875
8000 696.6207275390625
9000 696.59228515625
10000 696.5782470703125
11000 696.586181640625
12000 696.572998046875
13000 696.570068359375
14000 696.5711669921875
15000 696.568359375
16000 696.56591796875
17000 696.564941406

8000 696.58349609375
9000 696.585205078125
10000 696.586669921875
11000 696.5709228515625
12000 696.5706787109375
13000 696.5694580078125
14000 696.56787109375
15000 696.566162109375
16000 696.565185546875
17000 696.564697265625
18000 696.5648193359375
19000 696.5643310546875
20000 696.5643310546875
21000 696.5643310546875
22000 696.5640869140625
23000 696.5640869140625
24000 696.5640869140625
25000 696.5640869140625
26000 696.5640869140625
3.659021132965514e-06
5
20
0 9999999999
New best: 696.562255859375 696.562255859375
3256.66455078125 403.5 696.562255859375 44.59127426147461 24550
1000 697.289306640625
2000 697.1182861328125
3000 697.1751708984375
4000 697.03173828125
5000 696.727783203125
6000 696.768310546875
7000 696.6925048828125
8000 696.647705078125
9000 696.63916015625
10000 696.6070556640625
11000 696.59033203125
12000 696.577392578125
13000 696.57666015625
14000 696.5760498046875
15000 696.5706787109375
16000 696.569580078125
17000 696.5665283203125
18000 696.56494140625


1000 696.8543701171875
2000 697.01513671875
3000 696.9786376953125
4000 696.981689453125
5000 697.105224609375
6000 696.9727783203125
7000 696.8204345703125
8000 696.6812744140625
9000 696.6629638671875
10000 696.636474609375
11000 696.6087646484375
12000 696.5908203125
13000 696.59814453125
14000 696.575927734375
15000 696.57275390625
16000 696.5703125
17000 696.5692138671875
18000 696.5655517578125
19000 696.564697265625
20000 696.5640869140625
21000 696.5640869140625
22000 696.5638427734375
23000 696.5638427734375
24000 696.5638427734375
25000 696.5635986328125
26000 696.5635986328125
27000 696.5635986328125
28000 696.5635986328125
29000 696.5635986328125
4.870816258913356e-06
1
1
0 9999999999
New best: 696.562255859375 696.562255859375
3256.66455078125 403.5 696.562255859375 44.59127426147461 24550
1000 696.8436279296875
2000 697.0072021484375
3000 696.8148193359375
4000 696.6925048828125
5000 696.6536865234375
6000 696.5970458984375
7000 696.6058349609375
8000 696.5843505859375
90

1000 697.7191162109375
2000 697.1533203125
3000 697.5369873046875
4000 697.173828125
5000 696.936279296875
6000 696.8046875
7000 696.6483154296875
8000 696.66943359375
9000 696.6383056640625
10000 696.6044921875
11000 696.5946044921875
12000 696.59423828125
13000 696.5816650390625
14000 696.5758056640625
15000 696.570068359375
16000 696.5697021484375
17000 696.5667724609375
18000 696.5665283203125
19000 696.56494140625
20000 696.5645751953125
21000 696.5635986328125
22000 696.5635986328125
23000 696.5635986328125
24000 696.5635986328125
25000 696.5635986328125
26000 696.5635986328125
27000 696.563232421875
28000 696.563232421875
29000 696.563232421875
30000 696.563232421875
31000 696.563232421875
1.6627927320940476e-06
7
9
0 9999999999
New best: 696.562255859375 696.562255859375
3256.66455078125 403.5 696.562255859375 44.59127426147461 24550
1000 696.9722900390625
2000 696.95703125
3000 696.90234375
4000 696.7760009765625
5000 696.828125
6000 696.6314697265625
7000 696.6671142578125
80

4000 696.7728271484375
5000 696.75537109375
6000 696.6905517578125
7000 696.6676025390625
8000 696.614501953125
9000 696.6033935546875
10000 696.58056640625
11000 696.5789794921875
12000 696.5908203125
13000 696.5760498046875
14000 696.5687255859375
15000 696.5672607421875
16000 696.567626953125
17000 696.5653076171875
18000 696.5655517578125
19000 696.56494140625
20000 696.56494140625
21000 696.564697265625
22000 696.564697265625
23000 696.5645751953125
24000 696.5645751953125
25000 696.5635986328125
26000 696.5635986328125
27000 696.5635986328125
28000 696.5635986328125
29000 696.5635986328125
1.7383571428189658e-06
2
29
0 9999999999
New best: 696.5618896484375 696.5618896484375
3257.14013671875 403.5 696.5618896484375 44.622100830078125 24550
1000 697.4727783203125
2000 697.278564453125
3000 696.973388671875
4000 696.8245849609375
5000 696.74951171875
6000 696.7381591796875
7000 696.6279296875
8000 696.6551513671875
9000 696.607666015625
10000 696.5909423828125
11000 696.59375
12000

In [ ]:
best_ever_m64

In [ ]:
best_ever_m64[1][start_frame:end_frame]

In [ ]:
m64[1][10].stick_x

In [ ]:
game.read('gMarioState.slideYaw')

In [ ]:
game.read('gMarioState.vel')[1]

In [ ]:
m64[1][0]

In [ ]:
a = m64[1][0].copy()

In [ ]:
a.stick_x = 1

In [ ]:
a

In [ ]:
m64[1][0]